In [1]:
import sys

sys.path.append("..")
from cge_modeling import CGEModel, Variable, Parameter, Equation
from cge_modeling.base.cge import recursive_solve_symbolic, numba_linearize_cge_func
from cge_modeling.numba_tools import numba_lambdify
from cge_modeling.pytensorf.compile import *
from cge_modeling.pytensorf.optimize import root
from tests.utilities.models import load_model_1, load_model_2

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import sympy as sp

from sympytensor import as_tensor
from sympytensor.pymc import SympyDeterministic
import pytensor.tensor as pt
import pytensor
import pymc as pm

import arviz as az

In [2]:
mod = load_model_1(parse_equations_to_sympy=False)
f_model, f_jac, f_jac_inv = compile_cge_model_to_pytensor(mod)

In [3]:
for eq in mod.equations:
    print(eq.equation)

Y = A * K_d ** alpha * L_d ** (1 - alpha)
r * K_d = alpha * Y * P
w * L_d = (1 - alpha) * Y * P
income = w * L_s + r * K_s
C * P = income
L_s = L_d + resid
K_s = K_d
Y = C


In [4]:
mod.variable_names

['Y', 'L_d', 'K_d', 'C', 'income', 'r', 'P', 'resid']

In [5]:
mod.summary()

<IPython.core.display.Latex object>

In [7]:
data = np.array([11000, 7000, 4000, 11000, 11000, 1, 1, 0], dtype="float64")
data_scale = np.round(np.clip(np.log(data + 1e-8), 0.1, np.inf), 2)

In [9]:
n_eq = len(mod.unpacked_equation_names)
coords = {
    "equation": np.arange(n_eq, dtype=int),
    "equation_aux": np.arange(n_eq, dtype=int),
    "variable": mod.variable_names,
    "parameter": mod.parameter_names,
}

with pm.Model(coords=coords) as m:
    # Parameters
    A = pm.Gamma("A", 2, 1)
    alpha = pm.Beta("alpha", 3, 3)
    K_s = pm.Normal("K_s", 4000, 100)
    L_s = pm.Normal("L_s", 7000, 100)
    w = pt.ones((), dtype="float64")

    params = pm.Deterministic("params", pt.stack([alpha, A, L_s, K_s, w]), dims=["parameter"])

    x0 = pm.Normal("x0", mu=data, sigma=data_scale)
    #     x0 = pt.as_tensor(data, dtype='float64')

    root_history, converged, step_size, n_steps = root(f_model, f_jac_inv, x0=x0, exog=params)
    solution = pm.Deterministic("root", root_history[-1], dims=["variable"])
    error = pm.Deterministic("error", f_model(*solution, *params), dims=["equation"])
    success = pm.Deterministic("success", pt.allclose(error, 0))

    sigma = pm.HalfNormal("sigma", data_scale, dims=["variable"])
    jac = f_jac(*solution, *params)
    row_norm = (jac**2).sum(axis=1) ** 0.5
    normed_jac = jac / row_norm[:, None]

    rho = normed_jac @ normed_jac.T

    cov = pt.diag(sigma) @ rho @ pt.diag(sigma)
    cov = 0.5 * (cov + cov.T) + pt.identity_like(cov) * 1e-4
    chol = pt.linalg.cholesky(cov)
    #     y_hat = pm.Normal('y_hat', mu=solution, sigma=sigma, dims=['variable'], observed=data)
    y_hat = pm.MvNormal("y_hat", mu=solution, chol=chol, dims=["variable"], observed=data)

    pm.Potential("optimizer_failure", pt.switch(success, 0, -np.inf))

    #     idata = pm.sample(nuts_sampler='numpyro')
    idata = pm.sample_smc(
        kernel=pm.smc.MH,
        draws=1_000,
        correlation_threshold=0.5,
        threshold=0.99,
        chains=1,
        progressbar=True,
    )

Initializing SMC sampler...
Sampling 1 chain in 1 job


/Users/jessegrabowski/mambaforge/envs/cge-dev/lib/python3.11/site-packages/pymc/smc/kernels.py:462: RuntimeWarning: invalid value encountered in divide
  return np.abs(ab / (self.aa * bb))


KeyboardInterrupt: 

In [ ]:
az.plot_trace(idata, ["A", "alpha", "K_s", "L_s", "sigma"])

In [14]:
import pymc_experimental as pmx
from pymc_experimental.inference.smc.sampling import sample_smc_blackjax

In [ ]:
n_particles = 100
kernel = "NUTS"
inner_kernel_params = {"step_size": 0.1}
iterations_to_diagnose = 2

with m:
    idata = sample_smc_blackjax(
        n_particles=n_particles,
        kernel=kernel,
        inner_kernel_params=inner_kernel_params,
        iterations_to_diagnose=iterations_to_diagnose,
    )

In [15]:
idata

Inference data with groups:
	> posterior
	> observed_data